In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 4):  # There are 3 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('Telangana_bus_details.csv', index=False)

driver.quit()


Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/khammam-to-hyderabad: Message: 
Stacktrace:
0   chromedriver                        0x00000001026750b8 cxxbridge1$str$ptr + 1887276
1   chromedriver                        0x000000010266d794 cxxbridge1$str$ptr + 1856264
2   chromedriver                        0x000000010227c82c cxxbridge1$string$len + 88524
3   chromedriver                        0x00000001022c0834 cxxbridge1$string$len + 367060
4   chromedriver                        0x00000001022f848c cxxbridge1$string$len + 595500
5   chromedriver                        0x00000001022b5474 cxxbridge1$string$len + 321044
6   chromedriver                        0x00000001022b60e4 cxxbridge1$string$len + 324228
7   chromedriver                        0x000000010263ca9c cxxbridge1$str$ptr + 1656336
8   chromedriver                        0x00000001026414f8 cxxbridge1$str$ptr + 1675372
9   chromedriver                        0x0000000102622980 cxxbridge1$str$p

NameError: name 'driver' is not defined

In [2]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 8925,LAHARI A/C SLEEPER CUM SEATER,00:10,03h 55m,04:05,3.6,INR 389,18 Seats available
1,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 8987,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",00:25,03h 55m,04:20,4.3,INR 320,12 Seats available
2,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 8885,Lahari Non A/C Sleeper Cum Seater,01:15,04h 00m,05:15,4.9,INR 320,4 Seats available
3,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 8653,RAJADHANI AC (CONVERTED METRO LUXURY),03:15,04h 00m,07:15,4.2,INR 389,21 Seats available
4,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 8631,RAJDHANI (A.C. Semi Sleeper),04:00,04h 00m,08:00,3.7,INR 389,21 Seats available
...,...,...,...,...,...,...,...,...,...,...
1458,Hyderabad to Tirupati,https://www.redbus.in/bus-tickets/hyderabad-to...,Vandana Travels,NON A/C Sleeper (2+1),22:10,10h 50m,09:00,4.1,760,25 Seats available
1459,Hyderabad to Tirupati,https://www.redbus.in/bus-tickets/hyderabad-to...,Yolo Bus,Bharat Benz A/C Seater /Sleeper (2+1),18:30,11h 24m,05:54,2.8,INR 580,19 Seats available
1460,Hyderabad to Tirupati,https://www.redbus.in/bus-tickets/hyderabad-to...,Morning Star Travels,A/C Sleeper (2+1),20:00,11h 00m,07:00,2.3,INR 1290,30 Seats available
1461,Hyderabad to Tirupati,https://www.redbus.in/bus-tickets/hyderabad-to...,DHANUNJAYA TRAVELS,A/C Sleeper (2+1),18:00,12h 35m,06:35,1.4,INR 829,18 Seats available
